In [1]:
import random
import typing
from slippi_db import upload_lib

In [2]:
prod_meta = upload_lib.get_db('prod', 'meta')

In [3]:
%%time
meta = list(prod_meta.find({}))

In [4]:
meta_by_key = {info['key']: info for info in meta}

In [7]:
class Settings(typing.NamedTuple):
    num_invalid: int
    rest: int

In [8]:
def gather_keys(settings: Settings) -> typing.List[str]:
    infos = []

    # invalid
    invalid = list(filter(lambda info: info.get('invalid', False), meta))
    infos.extend(random.sample(invalid, settings.num_invalid))

    valid = list(filter(lambda info: not info.get('invalid', False), meta))
    infos.extend(random.sample(valid, settings.rest))

    keys = [info['key'] for info in infos]
    return keys

In [9]:
compression_test_settings = Settings(
    num_invalid=0,
    rest=10,
)

compression_test_keys = gather_keys(compression_test_settings)

In [ ]:
old_env = 'prod'

def make_new_env(new_env: str, keys: typing.List[str]):
    for db_name in [upload_lib.SLP]:
        old_db = upload_lib.get_db(old_env, db_name)
        new_db = upload_lib.get_db(new_env, db_name)
        
        infos = []
        for key in keys:
            info = old_db.find_one({'key': key})
            del info['_id']
            infos.append(info)

        new_db.insert_many(infos)
    
    for key in keys:
        source = dict(
            Bucket=upload_lib.bucket.name,
            Key='/'.join([old_env, upload_lib.SLP, key]),
        )
        
        upload_lib.bucket.copy(
            CopySource=source,
            Key='/'.join([new_env, upload_lib.SLP, key]),
        )

In [ ]:
make_new_env('compression_test', compression_test_keys)

In [ ]:
upload_lib.nuke_replays('compression_test', 'none')